In [2]:
import random
import numpy as np
import re
import pandas as pd
import os

In [3]:
llama_path = "./data/llama_embeddings"
llama_embs = []

In [12]:
for filename in os.listdir(llama_path):
    if filename.endswith(".npy"):
        file_path = os.path.join(llama_path, filename)
        data = np.load(file_path)
        llama_embs.append(data)

In [5]:
import os
import glob


npy_files = glob.glob(os.path.join(llama_path, "*.npy"))

output_txt_file = "references.txt"

with open(output_txt_file, "w") as txt_file:
    for npy_file in npy_files:
        file_name = os.path.basename(npy_file)
        file_name_without_extension = os.path.splitext(file_name)[0]
        txt_file.write(file_name_without_extension + "\n")

文件名已经存储在 references.txt 中。


In [13]:
refs = []

with open("references.txt", "r") as txt_file:
    for line in txt_file:
        refs.append(line.strip())


In [14]:
llama_embs_reshaped = [arr.reshape(-1) for arr in llama_embs]

In [15]:
from sklearn.cluster import KMeans
from sklearn import preprocessing
kmeans = KMeans(n_clusters = 2000, n_init = 1).fit_predict(preprocessing.normalize(llama_embs_reshaped))

In [17]:
import pandas as pd
tenders_structured_path = r"./data/UpdatedAgainTenders.xlsx"

tenders_structured = pd.read_excel(tenders_structured_path)
tenders_structured = tenders_structured[["Reference Number", "Contract Title", "Description"]].dropna(subset=["Reference Number"])

In [21]:
clusters = pd.DataFrame({"Reference Number": refs, "cluster": kmeans})
clusters = pd.merge(clusters, tenders_structured, on = "Reference Number")

In [22]:
from bs4 import BeautifulSoup
def remove_html_tags(text):
    soup = BeautifulSoup(text, "html.parser")
    cleaned = soup.get_text().replace('\xa0', ' ')
    cleaned = ' '.join(cleaned.split())
    return cleaned

docs = []
for index, row in clusters.iterrows():
    title = row["Contract Title"]
    desc = remove_html_tags(row["Description"])
    clusters.at[index, "Description"] = desc
    docs.append(title + ". " + desc)

In [25]:
clusters.loc[clusters["cluster"] == 300 , ["Reference Number", "Contract Title", "Description"]]

,Reference Number,Contract Title,Description
9530,ED210189,Provision of Probity Services for Program Kaar...,Provision of Probity Services for Program Kaar...
14461,HOU9617820,Maintenance of Housing Assets in the Ngaanyatj...,Maintenance of Housing Assets in the Ngaanyatj...
16348,LW2302,Probity auditor for gaming system renewal,Probity audit and ad hoc advice for procuremen...
19015,NMTAFEP2021021,Provision and Implementation of an Electronic ...,Provision and Implementation of an Electronic ...
23406,RFTMRWA001217STG6,Panel Contract for the Provision of Sprayed Bi...,Broome - Cape Leveque Road (BCLR) second-seals
24324,TWA2122136,Provision of Report on Tourism Investment Oppo...,Provision of Report on Tourism Investment Oppo...
